In [1]:
import Quandl
import numpy as np
import pandas as pd
import math
import scipy
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import os

%matplotlib inline  
pd.options.display.mpl_style = 'default'
pylab.rcParams['figure.figsize'] = 12, 6 

In [2]:
def get(ticker_list, kind = 'log_returns'):
    #takes in a ticker list and returns either a dataframe of closing price 'returns'
    #for each stock or closing 'prices'
    final_df = pd.DataFrame()
    for item in ticker_list:
        ticker = item
        temp_df = Quandl.get('YAHOO/'+str(ticker), 
                                         trim_start='2001-01-01', authtoken="H4uWGQ3jQdm96V7H2Abm")
        temp_df.index = pd.to_datetime(temp_df.index)
        temp_df = temp_df.sort()
        if (kind == 'prices'):
            add_series = temp_df.Close
        elif (kind == 'log_prices'):
            add_series = np.log(temp_df.Close);
        elif (kind == 'returns'):
            add_series = temp_df.Close.shift(-1) - temp_df.Close
        else:
            add_series = np.log(temp_df.Close).shift(-1) - np.log(temp_df.Close)
        final_df[ticker] = add_series
        

    final_df = final_df.sort()
    return final_df


In [30]:
def portfolio_sharpe(return_df, weight_list):
    risk = float(portfolio_risk(return_df, weight_list))
    weight_t = np.matrix(weight_list)
    mean = np.matrix(return_df.mean()).transpose()
    sharpe = weight_t * mean / risk
    

def portfolio_risk(return_df, weight_list):
    #Take in a PRICE_DF with columns being each stock's daily log_returns
    #Take in WEIGHT_LIST with each list entry corresponding to the respective price_df
    #column's weight in the portfolio
    #PRICE_DF should be a concatenation of return series from return_series()
    if (sum(weight_list) != 1):
        print("EROR: Weights do not sum to 1");
        return None
    cov_matrix = np.matrix(return_df.cov())
    weight_t = np.matrix(weight_list)
    weight = weight_t.transpose()
    variance = weight_t * cov_matrix * weight
    return variance
    

In [31]:
portfolio = get(['msft', 'mu', 'f', 'gs'])

In [32]:
(portfolio_sharpe(portfolio, [.2, .2, .4, .2]))
